In [1]:
import pandas
import synapseclient
import random
import csv

syn = synapseclient.login(silent=True)

In [2]:
def findAbstract(parentId, syn):
    r = syn.query('select id,name from folder where parentId=="%s"' % (parentId, ))
    r = filter(lambda x: x['folder.name'] == "Abstract", r['results'])

    if len(r) > 0:
        return r[0]['folder.id']
    else:
        return None

def get_abstract(projectId, syn):
    abstractId = findAbstract(projectId,  syn)
    w = syn.getWiki(abstractId)
    return w['markdown']

In [3]:
e = syn.getEvaluation(4484862)

In [4]:
sb = syn.getSubmissionBundles(e, status="VALIDATED")

In [5]:
abstracts = {x.id: get_abstract(x.entityId, syn) for (x, status) in sb}

In [6]:
nums = range(len(abstracts))
random.shuffle(nums)

In [7]:
def formatWiki(idnum, wMarkdown):
    return "## %s\n%s\n" % (idnum, wMarkdown)

abstractsWikiMarkdown = ""

for (n, (submissionId, wMarkdown)) in enumerate(abstracts.iteritems()):
    abstractsWikiMarkdown += formatWiki(nums[n], wMarkdown)
 

In [8]:
f = syn.store(synapseclient.Folder(name="Validated Submission Abstracts", parentId="syn4597738"))

try:
    w = syn.getWiki(owner=f)
except:
    w = synapseclient.Wiki(owner=f)

w.markdown = abstractsWikiMarkdown

w = syn.store(w)

In [9]:
with file("abstract_key_mapping.csv", "w") as of:
    wr = csv.writer(of)

    for (n, k) in enumerate(abstracts.keys()):
        wr.writerow([nums[n], k])

fmap = syn.store(synapseclient.File("abstract_key_mapping.csv", parentId=f.id))


##################################################
 Uploading file to Synapse storage 
##################################################
Uploading [####################]100.00%     170.0bytes/170.0bytes abstract_key_mapping.csv Done...
    